In [1]:
import numpy as np,numpy
import csv
import glob
import os

def dataimport(path1, path2):
    window_size = 1000
    threshold = 60
    slide_size = 200 #less than window_size!!!

	xx = np.empty([0,window_size,90],float)
	yy = np.empty([0,8],float)

	###Input data###
	#data import from csv
	input_csv_files = sorted(glob.glob(path1))
	for f in input_csv_files:
		print("input_file_name=",f)
		data = [[ float(elm) for elm in v] for v in csv.reader(open(f, "r"))]
		tmp1 = np.array(data)
		x2 =np.empty([0,window_size,90],float)
# 		print(tmp1.shape)

		#data import by slide window
		k = 0
		while k <= (len(tmp1) + 1 - 2 * window_size):
			x = np.dstack(np.array(tmp1[k:k+window_size, 1:91]).T)
# 			print(x.shape)
			x2 = np.concatenate((x2, x),axis=0)
# 			print(x2.shape)
			k += slide_size

		xx = np.concatenate((xx,x2),axis=0)
# 		print(xx.shape)
# 	xx = xx.reshape(len(xx),-1)
# 	print(xx.shape)


	###Annotation data###
	#data import from csv
	annotation_csv_files = sorted(glob.glob(path2))
	for ff in annotation_csv_files:
		print("annotation_file_name=",ff)
		ano_data = [[ str(elm) for elm in v] for v in csv.reader(open(ff,"r"))]
		tmp2 = np.array(ano_data)

		#data import by slide window
		y = np.zeros(((len(tmp2) + 1 - 2 * window_size)//slide_size+1,8))
		k = 0
		while k <= (len(tmp2) + 1 - 2 * window_size):
			y_pre = np.stack(np.array(tmp2[k:k+window_size]))
			bed = 0
			fall = 0
			walk = 0
			pickup = 0
			run = 0
			sitdown = 0
			standup = 0
			noactivity = 0
			for j in range(window_size):
				if y_pre[j] == "bed":
					bed += 1
				elif y_pre[j] == "fall":
					fall += 1
				elif y_pre[j] == "walk":
					walk += 1
				elif y_pre[j] == "pickup":
					pickup += 1
				elif y_pre[j] == "run":
					run += 1
				elif y_pre[j] == "sitdown":
					sitdown += 1
				elif y_pre[j] == "standup":
					standup += 1
				else:
					noactivity += 1

			if bed > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,1,0,0,0,0,0,0])
			elif fall > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,1,0,0,0,0,0])
			elif walk > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,1,0,0,0,0])
			elif pickup > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,0,1,0,0,0])
			elif run > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,0,0,1,0,0])
			elif sitdown > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,0,0,0,1,0])
			elif standup > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,0,0,0,0,1])
			else:
				y[int(k/slide_size),:] = np.array([2,0,0,0,0,0,0,0])
			k += slide_size

		yy = np.concatenate((yy, y),axis=0)
	print(xx.shape,yy.shape)
	return (xx, yy)

def split_dataset(x,y,dev_ratio,test_ratio):
	x_size = len(x)
	train_dev_size = int(x_size * (1-test_ratio))
	x_train_dev = x[:train_dev_size]
	x_test = x[train_dev_size:]
	y_train_dev = y[:train_dev_size]
	y_test = y[train_dev_size:]

	train_size = int(x_size * (1-dev_ratio-test_ratio))
# 	print(train_size)
	x_train = x_train_dev[:train_size]
# 	print(x_train.shape)
	x_dev = x_train_dev[train_size:]
	y_train = y_train_dev[:train_size]
# 	print(y_train.shape)
	y_dev = y_train_dev[train_size:]

	return x_train, x_dev, x_test, y_train, y_dev, y_test

#test
# for i, label in enumerate (["bed"]):
# 	filepath1 = "/home/yan/Dataset/Data/input_" + str(label) + "*.csv"
# 	filepath2 = "/home/yan/Dataset/Data/annotation_" + str(label) + "*.csv"

# 	x, y = dataimport(filepath1, filepath2)
# 	x_train, x_dev, x_test, y_train, y_dev, y_test = split_dataset(x,y,dev_ratio=0.1,test_ratio=0.1)

# xx = np.empty([0,1000,90],float)
# yy = np.empty([0,8],float)
# for i, label in enumerate (["bed", "walk", "run", "sitdown", "standup", "fall", "pickup"]):
# 	print(label,":")
# 	filepath1 = "/home/yan/work/Att_bi_lstm/data_test/input_" + str(label) + "*.csv"
# 	filepath2 = "/home/yan/work/Att_bi_lstm/data_test/annotation_" + str(label) + "*.csv"
# 	x, y = dataimport(filepath1, filepath2)
# 	print("x:", x.shape, "y:", y.shape)
# 	xx = np.concatenate((xx, x),axis=0)
# 	yy = np.concatenate((yy, y),axis=0)
# 	print(xx.shape, yy.shape)

# x_train, x_dev, x_test, y_train, y_dev, y_test = split_dataset(xx,yy,dev_ratio=0.1,test_ratio=0.1)
# print("x_train:", x_train.shape, "x_dev:", x_dev.shape, "x_test:", x_test.shape, "y_train:", y_train.shape, "y_test:", y_test.shape, "y_dev:", y_dev.shape)

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-1-277f22180ee0>, line 11)